In [1]:
import RNA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display

from collections import Counter
from collections import defaultdict
import subprocess
import difflib
import sys
import os
import random
import string
import time

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


2021-11-16 14:04:48.657277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-16 14:04:48.657291: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True) # no scientific notation

# import feature_generation
from features import ij_distance, new_move_dist, plt_moves, config_distance, balance_in_all_things, return_shift_moves
from process_features import fp_call, find_moves, process

In [3]:
sequence = 'AGGUGUUAUAAAAGUGCAGAGCCAAGUCACCGCCUUGCCGUCGCUUCCGCUAUGCCUCAAAAGCGGAAUAUCGAGCGAGCGAUUGUGAUUCGAAUGGUCG'
s1       = '.(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........'
s2       = '...(((.........)))..((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..'

search_width = 60
# search_width = 500

s = s1

sw = 2
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=sw, mp=True)
result = fp.get_en()/100.0
path = fp.get_path()


add_moves        = set()
delete_moves     = set()
available_add    = set()
available_delete = set()

en_contrib = []

for e, tup in enumerate(path):
    i = tup[0]
    j = tup[1]
    if i<0:
        delete_moves.add((i,j))
    if i>0:
        add_moves.add((i,j))

## we are evaluating 1 node = 1 structure
## means we are evaluating ALL moves (i, j) and see if we can find any patterns

##np.mean(en_contrib) ## mean of energies
##np.std(en_contrib) ## diversity of energies


## verhaeltnis delete add moves bezogen auf accessible moves
## mean of energy
## how many add / delete moves are currently available?
## if len(available_delete)/len(delete_moves) is high, the probability
## of making a wrong decision is probably quite high. 

## -------
## true values for classifier:
## - percentage of moves which lead to the optimal result?
## - sort by energy, where is the first contribution which leads to the optimal result? 
## - look at top energy contributions: how long do we have to postpone them to get the optimal result? 


fc = RNA.fold_compound(sequence)
pt = list(RNA.ptable(s))
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

for pos, (i,j) in enumerate(find_moves(pt, pt2)):  
    if (i,j) in add_moves:
        available_add.add((i,j))
    if (i,j) in delete_moves:
        available_delete.add((i,j))
    
    # map energies somehow to [0,1] such that the network will understand.  
    en = np.interp(fc.eval_move_pt(pt, i, j)/100, [-10,10], [0,1]) 
    en_contrib.append(en)

available_add, available_delete, len(available_add)/len(add_moves), len(available_delete)/len(delete_moves)

# np.std(en_contrib)

({(5, 17), (6, 16), (24, 95), (32, 83)},
 {(-43, -72),
  (-42, -73),
  (-41, -74),
  (-40, -76),
  (-39, -77),
  (-38, -79),
  (-37, -80),
  (-36, -81),
  (-35, -82),
  (-4, -21),
  (-3, -22),
  (-2, -23)},
 0.25,
 1.0)

In [4]:
# [   0,    0 ] .(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -23.10 0.125
# [  -4,  -21 ] .((..................)).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -20.70 0.1875
# [  -3,  -22 ] .(....................).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -17.70 0.0
# [  -2,  -23 ] ........................((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -21.90 0.05882
# [  23,   96 ] ......................(.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....)....  -18.10 0.0
# [  22,   97 ] .....................((.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....))...  -21.00 0.0
# [  24,   95 ] .....................(((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....)))...  -23.60 0.0
# [  21,   98 ] ....................((((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....))))..  -25.20 0.07692
# [ -35,  -82 ] ....................((((((((((.....((((((((.(((((((.(......).)))))))...))).)).)))...))))))....))))..  -23.60 0.08333
# [ -36,  -81 ] ....................((((((((((......(((((((.(((((((.(......).)))))))...))).)).))....))))))....))))..  -22.70 0.1667
# [ -37,  -80 ] ....................((((((((((.......((((((.(((((((.(......).)))))))...))).)).).....))))))....))))..  -19.10 0.0
# [ -38,  -79 ] ....................((((((((((........(((((.(((((((.(......).)))))))...))).)).......))))))....))))..  -20.00 0.1667
# [ -39,  -77 ] ....................((((((((((.........((((.(((((((.(......).)))))))...))).)........))))))....))))..  -17.40 0.0
# [ -40,  -76 ] ....................((((((((((..........(((.(((((((.(......).)))))))...)))..........))))))....))))..  -19.70 0.0
# [  32,   83 ] ....................((((((((((.(........(((.(((((((.(......).)))))))...)))........).))))))....))))..  -18.10 0.0
# [  33,   81 ] ....................((((((((((.((.......(((.(((((((.(......).)))))))...)))......).).))))))....))))..  -17.10 0.0
# [  34,   79 ] ....................((((((((((.(((......(((.(((((((.(......).)))))))...)))....).).).))))))....))))..  -16.30 0.0
# [  35,   78 ] ....................((((((((((.((((.....(((.(((((((.(......).)))))))...)))...)).).).))))))....))))..  -17.90 0.0
# [  36,   77 ] ....................((((((((((.(((((....(((.(((((((.(......).)))))))...)))..))).).).))))))....))))..  -19.50 0.0
# [  37,   76 ] ....................((((((((((.((((((...(((.(((((((.(......).)))))))...))).)))).).).))))))....))))..  -21.50 0.0
# [  38,   75 ] ....................((((((((((.(((((((..(((.(((((((.(......).)))))))...)))))))).).).))))))....))))..  -24.60 0.0
# [ -41,  -74 ] ....................((((((((((.(((((((...((.(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.20 0.0
# [ -42,  -73 ] ....................((((((((((.(((((((....(.(((((((.(......).)))))))...)..))))).).).))))))....))))..  -19.90 0.0
# [ -43,  -72 ] ....................((((((((((.(((((((......(((((((.(......).)))))))......))))).).).))))))....))))..  -22.70 0.0
# [  39,   73 ] ....................((((((((((.((((((((.....(((((((.(......).)))))))....).))))).).).))))))....))))..  -21.90 0.0
# [  40,   72 ] ....................((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -25.20 0.0
# [   5,   17 ] ....(...........)...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -21.20 0.0
# [   6,   16 ] ....((.........))...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.70 0.0

In [5]:
def adjust_pt(pt, i, j):
    pt_adj = pt.copy()
    if i<0:
        pt_adj[-i] = 0
        pt_adj[-j] = 0
    else:
        pt_adj[i] = j
        pt_adj[j] = i
    return pt_adj

In [33]:
reloaded_model = tf.keras.models.load_model('dnn_model')

data = {'en_mean': [0.678000],
        'en_std': 0.0328022,
        'best_en': 0.625,
        'vic': 0,
        'vic_best': 0,
        'unique_moves': 1,
        'distlast': 0.130}
test_features = pd.DataFrame.from_dict(data)


y = reloaded_model.predict(test_features)[0][0]

y

-0.0599702

In [7]:
from sklearn.cluster import DBSCAN

moves = [(-78, -90, 80), (-80, -89, 90), (-5, -51, 130), (-58, -71, 170), (-61, -68, 170), (-8, -48, 230), (-76, -92, 250), (-75, -96, 270), (-72, -99, 300), (-60, -69, 420), (-6, -50, 500), (-73, -98, 500), (-59, -70, 620), (-77, -91, 620), (-7, -49, 640), (-74, -97, 640)]
moves = [(52, 90, 360), (47, 95, 510), (51, 91, 600), (50, 92, 610), (49, 93, 620), (48, 94, 650), (46, 96, 680)]

moves = np.array([(abs(i[0]), abs(i[1])) for i in moves])

# X = np.array([[1, 2], [2, 2], [2, 3],
            #   [8, 7], [8, 8], [25, 80]])

clustering = DBSCAN(eps=2, min_samples=1).fit(moves)

for i in range(len(moves)):
    print (moves[i], clustering.labels_[i])

len(set(clustering.labels_))

[52 90] 0
[47 95] 0
[51 91] 0
[50 92] 0
[49 93] 0
[48 94] 0
[46 96] 0


1

In [27]:

sequence = 'CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAGAGCUCAAGAUUCAAACGGCGGCUUCAGAACUCAGUCAGGCGAAGAAGCCAGCAC'
s1       = '.......((.(.(.((((((....)))))).).)...))........................((((((((....((......))....)))))).))..'
s2       = '.(((.((((..(..((((((....)))))).).........)))))))...............((((((((....((......))....)))))).))..'

# sequence = 'GAGACCGAUACGAUUAAGCCAGAAUUCCAAUGAUUUGGGGAUCGGGGCUGUUUAGUCUCGUUUAAUCCCGGCGAUGUCUGGGUCAUAUUGUCCGUUAACC' 
# s1       = '(.((.(((((.(((....(((((..(((((....)))(((..(((((((....)))))))......)))...))..)))))))).)))))))).......'
# s2       = '.((((.((.((.....((((.((.((((((....)))))).))..)))).....)).))))))...(((((......)))))..................'

sequence = 'GACUGCCUCAAUUUCCACUUCGCCACAAUGUGAGGUCUCAUUCAACUGCCGCUCAUAAUGGUCGGUCAAUGAGGUCCAUCCGGUCCCUCCAGCCUGGCUA'
s1       = '.....................((((...((..((..((((((..((((((.........)).)))).))))))............).).))...))))..'
s2       = '.....................((((...((.((((.((...(((...((((((......)).))))...)))((.....)))).)).))))...))))..'



# alternative 192
# 807 ! 809
filename_samples = f'./dataset_100_large.csv'
samples_df = pd.read_csv(filename_samples)
for index, row in samples_df.iterrows():
    if index != 13:
        continue
    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2

print(f"sequence = \"{sequence}\"")
print(f"s1 = \"{s1}\"")
print(f"s2 = \"{s2}\"")
print (RNA.bp_distance(s1, s2))


fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=sw, mp=False)
result = fp.get_en()/100.0
path = fp.get_path()

# path = [(0, 0, -2330), (-80, -91, -2170), (-81, -90, -1880), (-82, -89, -2120), (-13, -62, -1940), (-12, -63, -2100), (-4, -75, -1750), (-5, -74, -1730), (-6, -73, -1640), (-7, -72, -1500), (-8, -71, -1500), (-10, -69, -1170), (-11, -68, -1620), (74, 81, -1290), (75, 80, -1600), (4, 11, -1320), (-15, -59, -1300), (3, 12, -1490), (5, 10, -1630), (-16, -58, -1250), (-17, -57, -1020), (-18, -56, -870), (-19, -55, -770), (-20, -54, -900), (17, 97, -490), (18, 96, -780), (19, 95, -900), (16, 98, -1040), (15, 99, -1140), (-21, -52, -790), (21, 94, -710), (-24, -49, -540), (-22, -51, -790), (-25, -48, -690), (24, 91, -460), (25, 90, -610), (-26, -47, -370), (-27, -46, -370), (26, 89, -650), (27, 88, -850), (28, 87, -980), (-34, -39, -770), (-30, -43, -640), (-33, -40, -470), (-31, -42, -430), (-32, -41, -465), (38, 66, -370), (37, 67, -480), (29, 86, -820), (46, 54, -420), (36, 68, -630), (45, 55, -810), (39, 61, -640), (47, 53, -710), (40, 60, -810), (42, 58, -900), (44, 56, -1050), (41, 59, -1660)]
# path = [(0, 0, -2330), (-80, -91, -2170), (-81, -90, -1880), (-82, -89, -2120), (-13, -62, -1940), (-12, -63, -2100), (-4, -75, -1750), (-5, -74, -1730), (-6, -73, -1640), (-7, -72, -1500), (-8, -71, -1500), (-10, -69, -1170), (-11, -68, -1620), (74, 81, -1290), (75, 80, -1600), (4, 11, -1320), (-15, -59, -1300), (3, 12, -1490), (5, 10, -1630), (-16, -58, -1250), (-17, -57, -1020), (-18, -56, -870), (-19, -55, -770), (-20, -54, -900), (17, 97, -490), (18, 96, -780), (19, 95, -900), (16, 98, -1040), (15, 99, -1140), (-21, -52, -790), (21, 94, -710), (-24, -49, -540), (-22, -51, -790), (-25, -48, -690), (24, 91, -460), (25, 90, -610), (-26, -47, -370), (-27, -46, -370), (26, 89, -650), (27, 88, -850), (28, 87, -980), (-30, -43, -640), (-34, -39, -770), (-33, -40, -470), (-31, -42, -430), (-32, -41, -465), (38, 66, -370), (37, 67, -480), (29, 86, -820), (46, 54, -420), (36, 68, -630), (45, 55, -810), (39, 61, -640), (47, 53, -710), (40, 60, -810), (42, 58, -900), (44, 56, -1050), (41, 59, -1660)]

print_moves(sequence, s1, s2, path)

sequence = "UGCAAUCCUGCCUAUCGAAAGCUCUGAAUUAAAUGAUGAAGGCAGCUCCUUAUUUAAAGUACGUUUACCGUCGGGAAUCUCUCAACAGACGAUUAGCCGC"
s1 = ".......(((((((((..................))))..))))).........((((.....))))....(((.(((((((....))).))))..)))."
s2 = ".((.....(((((..........................)))))))......................((((.(...........).))))........."
27
UGCAAUCCUGCCUAUCGAAAGCUCUGAAUUAAAUGAUGAAGGCAGCUCCUUAUUUAAAGUACGUUUACCGUCGGGAAUCUCUCAACAGACGAUUAGCCGC
[   0,    0 ] .......(((((((((..................))))..))))).........((((.....))))....(((.(((((((....))).))))..))).  -1190.00
[ -13,  -38 ] .......(((((.(((..................)))...))))).........((((.....))))....(((.(((((((....))).))))..))).  -1120.00
[  13,   40 ] .......(((((((((..................)))..)))))).........((((.....))))....(((.(((((((....))).))))..))).  -1250.00
[ -55,  -67 ] .......(((((((((..................)))..))))))..........(((.....))).....(((.(((((((....))).))))..))).  -1130.00
[ -56,  -66 ] .......(((((((((..................)))..)))))).......

-470

In [34]:
# which nodes do we permutate first? in principle, this is a simlar concept as the findpath search width, where we just
# use the sw. to perform a look-ahead

## IDEA
# check how many moves out of all choices are net-negative. 

search_width_multiplier = 6



fc = RNA.fold_compound(sequence)
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

# for i,j in find_moves(pt1, pt2):    
#     pt = adjust_pt(pt1, i, j)
#     s = RNA.db_from_ptable(pt)
#     search_width = 1
#     sE, path = findpath_librna.pathfinder(sequence, s, s2, search_width=search_width, return_paths=True)
#     en = fc.eval_move_pt(pt1, i, j)
#     # print_moves(sequence, s, s2, path)
#     print (s, i, j, en, sE)


def next_vicinity(lastmove, avail_moves):

    """
    percentage of moves which can continue in the vicinity of the last move
    """
    lasti, lastj = lastmove
    moves = len(avail_moves)
    cntr = 0

    for a in avail_moves:
        i=a[0]
        j=a[1]      
        if lasti + 1 == i or lasti - 1 == i or\
           lastj + 1 == j or lastj - 1 == j:
           cntr += 1
    return cntr


def last_vs_best(lastmove, avail_moves, lensequence):
    lasti = abs(lastmove[0])
    lastj = abs(lastmove[1])
    i = abs(avail_moves[0][0])
    j = abs(avail_moves[0][1])

    dist = abs(lasti - i) + abs(lastj - j)
    return dist/(lensequence*2) #, dist, lensequence


def cluster_moves(moves):
    moves = np.array([(abs(i[0]), abs(i[1])) for i in moves])
    clustering = DBSCAN(eps=2, min_samples=1).fit(moves)
    unique_labels = len(set(clustering.labels_))
    # normalize to 1
    return 1/unique_labels


# def structure_evaluation(sequence, s, s2):
def structure_evaluation(fc, pt1, pt2, path, lastmove):
    avail_moves = []
    fp_results = []
    en_contrib = []

    available_add = set()
    available_delete = set()
    lensequence = pt1[0]

    # pt1 = list(RNA.ptable(s))
    # pt2 = list(RNA.ptable(s2))
    for pos, (i,j) in enumerate(find_moves(pt1, pt2)):    
        next_en = fc.eval_move_pt(pt1, i, j)
        # mark where we found our move    
        # map energies somehow to [0,1] such that the network will understand.  
        en = np.interp(next_en/100, [-10,10], [0,1]) 
        en_contrib.append(en)
        avail_moves.append((i, j, next_en))

        if i>0:
            available_add.add((i,j))
        elif i<0:
            available_delete.add((i,j))

    if len(avail_moves)==0:
        add_delete = 0
    else:
        add_delete = len(available_delete)/len(avail_moves)

    # print ('best', en_contrib[0], 'avg', en_mean)

    avail_moves.sort(key=lambda x: x[2])


    vic = next_vicinity(lastmove, avail_moves)
    vic_best = next_vicinity(lastmove, avail_moves[0:1])

    distlast = last_vs_best(lastmove, avail_moves, lensequence)

    unique_moves = cluster_moves(avail_moves)
    count_available = len(avail_moves)/len(path)

    # print ("vicinity", vic, lastmove, "->", avail_moves)

    # print ('avail add', available_add)
    # print ('avail del', available_delete)

    for pos, (i,j, en) in enumerate(avail_moves):  
        pt = adjust_pt(pt1, i, j)
        snew = RNA.db_from_ptable(pt)
        # if i > 0:
        #     snew = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
        # if i < 0:
        #     snew = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]

        ptnew = list(RNA.ptable(snew))
        result_new, path = fp_call(sequence, snew, s2, search_width_multiplier)
        fp_results.append(result_new)

    if len(avail_moves) == 0:
        label = 0
        en_mean = 0
        en_std = 0
        best_en = 0


    else:
        label = np.argmin(fp_results)/len(fp_results)

        en_mean = np.mean(en_contrib)
        en_std = np.std(en_contrib)
        en_contrib.sort(key=lambda x: x)
        en_diff = en_contrib[0] / en_mean # best en, relative

        best_en = en_contrib[0]




    # print (avail_moves)
    # print (fp_results)
    # print (avail_moves, unique_moves)
    
    return label, en_mean, en_std, best_en, vic, vic_best, unique_moves, distlast

    # return label, en_mean, en_diff, avail_moves[0:3]

    return label, en_mean, en_diff, avail_moves[0:3], best_en

    print (fp_results)
    print (best_result)

# structure_evaluation(sequence, s, s2)
# structure_evaluation(sequence, pt1, pt2)


fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=sw, mp=False)
result = fp.get_en()/100.0
path = fp.get_path()


pt = pt1.copy()
lastmove = 0, 0
lastcount_available = 0

labels = []

for pos, (i, j, en) in enumerate(path):
    if i==0: 
        print (RNA.db_from_ptable(pt)[0:100])
        continue
    # if i not in random_moves:
    #     continue

    fc = RNA.fold_compound(sequence)
    label, en_mean, en_std, best_en, vic, vic_best, unique_moves, distlast = structure_evaluation(fc, pt, pt2, path[1:], lastmove)
    current_en = round(fc.eval_structure_pt(pt)/100,2)


    # compare with prediction
    data = {'en_mean': [en_mean],
            'en_std': en_std,
            'best_en': best_en,
            'vic': vic,
            'vic_best': vic_best,
            'unique_moves': unique_moves,
            'distlast': distlast}
    test_features = pd.DataFrame.from_dict(data)  
    y = reloaded_model.predict(test_features)[0][0]
    pred_label = round(y,4)

    # display_count = lastcount_available - count_available
    pt = adjust_pt(pt, i, j)

    labels.append(label)
    vic = round(vic, 3)
    vic_best = round(vic_best, 3)
    en_mean = round(en_mean, 3)
    en_std = round(en_std, 3)
    best_en = round(best_en, 3)

    print (RNA.db_from_ptable(pt)[0:100], current_en, 'label', round(label,4), 'pred', pred_label, (i,j), en_mean, en_std, best_en, vic, vic_best, unique_moves, distlast)
    # print (RNA.db_from_ptable(pt)[0:100], i, j, best_result)
    
    lastmove = i, j
    # lastcount_available = count_available

    # ptlast = pt

    # if pos == 5:
    #     break


print (labels)

# print_moves(sequence, s1, s2, path)


# while loop - increase sw from 1 onwards until we find the optimal path. how easy is the next decision?



# distance of best move in relation to last move
# moves left in vicinity, which are not the best moves. 
# !! consider number of avail moves / +before difference comparison...

.......(((((((((..................))))..))))).........((((.....))))....(((.(((((((....))).))))..))).
.......(((((.(((..................)))...))))).........((((.....))))....(((.(((((((....))).))))..))). -11.9 label 0.0 pred 0.0848 (-13, -38) 0.667 0.081 0.535 0 0 0.14285714285714285 0.255
.......(((((((((..................)))..)))))).........((((.....))))....(((.(((((((....))).))))..))). -11.2 label 0.0 pred -0.0045 (13, 40) 0.657 0.092 0.435 1 0 0.125 0.01
.......(((((((((..................)))..))))))..........(((.....))).....(((.(((((((....))).))))..))). -12.5 label 0.0526 pred 0.0896 (-55, -67) 0.666 0.081 0.55 0 0 0.14285714285714285 0.02
.......(((((((((..................)))..))))))...........((.....))......(((.(((((((....))).))))..))). -11.3 label 0.0556 pred 0.0846 (-56, -66) 0.663 0.084 0.55 1 0 0.14285714285714285 0.355
.......(((((((((..................)))..))))))............(.....).......(((.(((((((....))).))))..))). -10.3 label 0.0 pred 0.0004 (-57, -65) 0.661 0.086 0.545 1 

In [1]:
# sE, path = fp_call(sequence, s1, s2, search_width_multiplier)
# print_moves(sequence, s1, s2, path)

In [26]:
filename_samples = f'./dataset_100_large.csv'
samples_df = pd.read_csv(filename_samples)
sequence = ''
s1 = ''
s2 = ''
search_width_multiplier = 6

df = pd.DataFrame()
all_results = []

for index, row in samples_df.iterrows():
    # if index != 90:
    #     continue


    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2


    bp_dist = RNA.bp_distance(s1, s2)
    # print (sequence)
    # print (s1)
    # print (s2)
    # print (bp_dist)

    # random_move = random.choice(range(bp_dist))

    fc = RNA.fold_compound(sequence)
    pt1 = list(RNA.ptable(s1))
    pt2 = list(RNA.ptable(s2))
    # structure_evaluation(sequence, pt1, pt2)

    # sE, path = fp_call(sequence, s1, s2, search_width_multiplier)
    fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=False)
    sE = fp.get_en()/100.0
    path = fp.get_path()

    # print (path)
    path = path[0:-10] # ignore the last moves, we're not interested in meaningless decisions
    # print (path)

    random_moves = random.choices([i[0] for i in path], k=3)
    # print (random_moves)
    # print (len(path))

    ptlast = list(RNA.ptable(s1))
    lastmove = 0, 0

    all_evals = 0
    results = []

    # find a random structure in the path
    for i, j, en in path:
        if i==0: 
            continue
        # if i not in random_moves:
        #     continue

        # print (i, RNA.db_from_ptable(pt))

        if i in random_moves:
            label, en_mean, en_std, best_en, vic, vic_best, unique_moves, distlast = structure_evaluation(fc, pt, pt2, path, lastmove)
            results.append((sequence, RNA.db_from_ptable(pt), s2, i, j, label, en_mean, en_std, best_en, vic, vic_best, unique_moves, distlast))
            all_evals += label       
            # print (i, eval)  

        pt = adjust_pt(ptlast, i, j)

        lastmove = i, j
        ptlast = pt

    if all_evals != 0:
        all_results += results


    # break

    if index%10 == 0:        
        print (index, end=" ")
        # print (all_results)
        df = pd.DataFrame(all_results, columns=['sequence', 's', 's2', 'i', 'j', 'target', 'en_mean', 'en_std', 'best_en', 'vic', 'vic_best', 'unique_moves', 'distlast'])# .set_index('sequence')
        filename = f'./move_diversity3.csv'
        df.to_csv(filename)

    if index == 800:
        break



0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 

## tf part